<a href="https://colab.research.google.com/github/qKTPq/2110574-AI4ENG/blob/main/KTP_Informed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Star 8 Puzzles

In [ ]:
# --- A* for 8-puzzle using h1 (misplaced tiles) ---

# Encoding:
# state is a 10-char string: state[0] = index (0..8) of the blank in the 3x3 board,
# state[1:] = the 9 chars of the board row-major including '0' for the blank.
# Example: goal_state = '8123456780' means blank at index 8 and board '123456780'.

from collections import deque

# --------- Config (choose one start state) ----------
goal_state = '8123456780'   # blank at index 8, board "123456780"
start_state = '2170532486'  # chosen start (same for both cells)
# ---------------------------------------------------

goal_puzzle = goal_state[1:]  # '123456780'

# Heuristic h1: Misplaced tiles (excluding blank)
def h1(state: str) -> int:
    p = state[1:]
    count = 0
    for i in range(9):
        if p[i] == '0':
            continue
        if p[i] != goal_puzzle[i]:
            count += 1
    return count

# Pretty print helper
def print_board(state: str):
    p = state[1:]
    print(p[0:3])
    print(p[3:6])
    print(p[6:9])
    print()

# Generate successors for a node using current heuristic function
# Node tuple: (state, node_id, parent_id, depth, g_cost, h)
last_index = 0
def gen_successors(node, heuristic):
    global last_index
    state, node_id, parent_id, depth, g_cost, h_val = node
    loc = int(state[0])
    p = list(state[1:])

    neighbors = []
    moves = []
    # valid moves for blank index 'loc'
    if loc % 3 != 0:      # left
        moves.append(loc - 1)
    if (loc + 1) % 3 != 0:  # right
        moves.append(loc + 1)
    if loc >= 3:          # up
        moves.append(loc - 3)
    if loc <= 5:          # down
        moves.append(loc + 3)

    for new_loc in moves:
        p2 = p[:]
        p2[loc], p2[new_loc] = p2[new_loc], p2[loc]
        new_state = str(new_loc) + ''.join(p2)
        g2 = g_cost + 1
        h2 = heuristic(new_state)
        last_index += 1
        neighbors.append((new_state, last_index, node_id, depth + 1, g2, h2))
    return neighbors

def is_goal(state: str) -> bool:
    return state == goal_state

# A* search with a chosen heuristic; returns goal node and stats
def a_star(start_state: str, heuristic):
    import heapq

    global last_index
    last_index = 0

    # Counters
    total_generated = 0
    total_expanded = 0

    # Priority queue of (f, tie_breaker, node)
    tie = 0
    start_node = (start_state, 0, -1, 0, 0, heuristic(start_state))
    fringe = [(start_node[4] + start_node[5], tie, start_node)]
    tie += 1

    # For path reconstruction by node_id
    visited_by_id = {0: start_node}
    # Closed set of states to avoid re-expansion
    closed = set()

    while fringe:
        f, _, node = heapq.heappop(fringe)
        state, node_id, parent_id, depth, g_cost, h_val = node

        if state in closed:
            continue
        closed.add(state)
        total_expanded += 1

        if is_goal(state):
            # Print the path
            print("Start state:")
            print_board(start_state)
            print("Goal path:")
            cur = node
            while True:
                print_board(cur[0])
                if cur[2] == -1:
                    break
                cur = visited_by_id[cur[2]]
            return {
                "goal_node": node,
                "generated": total_generated,
                "expanded": total_expanded,
                "depth": depth
            }

        # Expand
        children = gen_successors(node, heuristic)
        total_generated += len(children)  # count generated successors
        for child in children:
            c_state, c_id, c_parent, c_depth, c_g, c_h = child
            visited_by_id[c_id] = child
            if c_state in closed:
                continue
            heapq.heappush(fringe, (c_g + c_h, tie, child))
            tie += 1

    return {"goal_node": None, "generated": total_generated, "expanded": total_expanded, "depth": None}

# --------- Run A* with h1 ----------
result_h1 = a_star(start_state, h1)
print("Heuristic: h1 (misplaced tiles)")
print(f"Generated nodes: {result_h1['generated']}")
print(f"Expanded nodes:  {result_h1['expanded']}")
print(f"Solution depth:  {result_h1['depth']}")

# --- A* for 8-puzzle using h2 (Manhattan distance) ---

from collections import deque

# --------- Same config (must match Cell 1) ----------
goal_state = '8123456780'
start_state = '2170532486'
# ---------------------------------------------------

goal_puzzle = goal_state[1:]  # '123456780'

# Heuristic h2: total Manhattan distance (excluding blank)
def h2(state: str) -> int:
    p = state[1:]
    dist = 0
    for i in range(9):
        t = p[i]
        if t == '0':
            continue
        j = goal_puzzle.index(t)
        ri, ci = divmod(i, 3)
        rj, cj = divmod(j, 3)
        dist += abs(ri - rj) + abs(ci - cj)
    return dist

def print_board(state: str):
    p = state[1:]
    print(p[0:3])
    print(p[3:6])
    print(p[6:9])
    print()

# Node tuple: (state, node_id, parent_id, depth, g_cost, h)
last_index = 0
def gen_successors(node, heuristic):
    global last_index
    state, node_id, parent_id, depth, g_cost, h_val = node
    loc = int(state[0])
    p = list(state[1:])

    neighbors = []
    moves = []
    if loc % 3 != 0:         # left
        moves.append(loc - 1)
    if (loc + 1) % 3 != 0:   # right
        moves.append(loc + 1)
    if loc >= 3:             # up
        moves.append(loc - 3)
    if loc <= 5:             # down
        moves.append(loc + 3)

    for new_loc in moves:
        p2 = p[:]
        p2[loc], p2[new_loc] = p2[new_loc], p2[loc]
        new_state = str(new_loc) + ''.join(p2)
        g2 = g_cost + 1
        h2v = heuristic(new_state)
        last_index += 1
        neighbors.append((new_state, last_index, node_id, depth + 1, g2, h2v))
    return neighbors

def is_goal(state: str) -> bool:
    return state == goal_state

def a_star(start_state: str, heuristic):
    import heapq

    global last_index
    last_index = 0

    total_generated = 0
    total_expanded = 0

    tie = 0
    start_node = (start_state, 0, -1, 0, 0, heuristic(start_state))
    fringe = [(start_node[4] + start_node[5], tie, start_node)]
    tie += 1

    visited_by_id = {0: start_node}
    closed = set()

    while fringe:
        f, _, node = heapq.heappop(fringe)
        state, node_id, parent_id, depth, g_cost, h_val = node

        if state in closed:
            continue
        closed.add(state)
        total_expanded += 1

        if is_goal(state):
            print("Start state:")
            print_board(start_state)
            print("Goal path:")
            cur = node
            while True:
                print_board(cur[0])
                if cur[2] == -1:
                    break
                cur = visited_by_id[cur[2]]
            return {
                "goal_node": node,
                "generated": total_generated,
                "expanded": total_expanded,
                "depth": depth
            }

        children = gen_successors(node, heuristic)
        total_generated += len(children)
        for child in children:
            c_state, c_id, c_parent, c_depth, c_g, c_h = child
            visited_by_id[c_id] = child
            if c_state in closed:
                continue
            heapq.heappush(fringe, (c_g + c_h, tie, child))
            tie += 1

    return {"goal_node": None, "generated": total_generated, "expanded": total_expanded, "depth": None}

# --------- Run A* with h2 ----------
result_h2 = a_star(start_state, h2)
print("Heuristic: h2 (Manhattan distance)")
print(f"Generated nodes: {result_h2['generated']}")
print(f"Expanded nodes:  {result_h2['expanded']}")
print(f"Solution depth:  {result_h2['depth']}")

Start state:
170
532
486

Goal path:
123
456
780

123
456
708

123
456
078

123
056
478

123
506
478

123
576
408

123
576
480

123
570
486

120
573
486

102
573
486

172
503
486

172
530
486

170
532
486

Heuristic: h1 (misplaced tiles)
Generated nodes: 232
Expanded nodes:  87
Solution depth:  12
Start state:
170
532
486

Goal path:
123
456
780

123
456
708

123
456
078

123
056
478

123
506
478

123
576
408

123
576
480

123
570
486

120
573
486

102
573
486

172
503
486

172
530
486

170
532
486

Heuristic: h2 (Manhattan distance)
Generated nodes: 95
Expanded nodes:  36
Solution depth:  12


In [ ]:
import random
random.seed(42)

def gen_start(state, step):
    for _ in range(step):
        loc0 = int(state[0])
        dir = random.randint(0,3)
        if dir == 0 and loc0 >= 3:
            new_loc0 = loc0 - 3
        elif dir == 1 and loc0 <= 5:
            new_loc0 = loc0 + 3
        elif dir == 2 and loc0 % 3 != 0:
            new_loc0 = loc0 - 1
        elif dir == 3 and (loc0 + 1) % 3 != 0:
            new_loc0 = loc0 + 1
        else:
            new_loc0 = loc0
        newstate = list(state[1:])
        newstate[loc0], newstate[new_loc0] = newstate[new_loc0], newstate[loc0]
        state = str(new_loc0) + ''.join(newstate)
    return state

goal_state = '8123456780'
start_state = gen_start(goal_state, 60)
print(start_state)  # copy this into both cells

3123056478
